In [9]:
import numpy as np
import pandas as pd

# 1) Define danger‐component functions
def compute_ball_factor(ball_pos, goal_pos=(100, 50)):
    dist_ball = np.linalg.norm(np.array(ball_pos) - np.array(goal_pos))
    max_dist  = np.linalg.norm(np.array([0, 0]) - np.array(goal_pos))
    return float(np.clip(1 - (dist_ball / max_dist), 0, 1))

def compute_pressure(players, attacking_team, ball_pos, radius=20):
    atk_near = 0
    def_near = 0
    for (x, y), team in players:
        if np.linalg.norm(np.array((x, y)) - np.array(ball_pos)) < radius:
            if team == attacking_team:
                atk_near += 1
            else:
                def_near += 1
    total_near = atk_near + def_near
    return float(atk_near / total_near) if total_near > 0 else 0.0

def compute_audience_factor(audience_loudness, max_loudness=120):
    return float(np.clip(audience_loudness / max_loudness, 0, 1))

def compute_danger(ball_pos, players, attacking_team, audience_loudness,
                   goal_pos=(100, 50), radius=20,
                   w_ball=0.7, w_pressure=0.2, w_audience=0.3):
    bf = compute_ball_factor(ball_pos, goal_pos)
    pr = compute_pressure(players, attacking_team, ball_pos, radius)
    af = compute_audience_factor(audience_loudness)
    return float(np.clip(w_ball * bf + w_pressure * pr + w_audience * af, 0, 1))


In [11]:

# 2) Load snapshots with updated path
csv_path = '/content/Random_Snapshots_CSV_Preview.csv' # as
df = pd.read_csv(csv_path)

# 3) Iterate and print danger with loudness details
print("Attack Danger Monitoring with Loudness\n" + "-"*40)
for idx, row in df.iterrows():
    ball_pos      = (row['ball_x'], row['ball_y'])
    audience_dB   = row['audience_dB']
    audience_fac  = compute_audience_factor(audience_dB)
    players = [
        ((row[f'player{i}_x'], row[f'player{i}_y']), row[f'player{i}_team'])
        for i in range(1, 23)
    ]
    score = compute_danger(ball_pos, players, attacking_team='Home',
                           audience_loudness=audience_dB, radius=15)
    status = "HIGH DANGER ⚠️ [Take Pictures]" if score >= 0.85 else "safe"
    print(f"Snapshot {idx+1:2d}: Danger={score:.2f}, "
          f"Loudness={audience_dB:.1f}dB (Factor={audience_fac:.2f}) → {status}")


Attack Danger Monitoring with Loudness
----------------------------------------
Snapshot  1: Danger=0.68, Loudness=90.1dB (Factor=0.75) → safe
Snapshot  2: Danger=0.46, Loudness=60.0dB (Factor=0.50) → safe
Snapshot  3: Danger=0.66, Loudness=87.5dB (Factor=0.73) → safe
Snapshot  4: Danger=0.78, Loudness=74.5dB (Factor=0.62) → safe
Snapshot  5: Danger=1.00, Loudness=108.6dB (Factor=0.91) → HIGH DANGER ⚠️ [Take Pictures]
Snapshot  6: Danger=0.54, Loudness=70.3dB (Factor=0.59) → safe
Snapshot  7: Danger=0.83, Loudness=101.1dB (Factor=0.84) → safe
Snapshot  8: Danger=0.30, Loudness=81.9dB (Factor=0.68) → safe
Snapshot  9: Danger=0.65, Loudness=103.5dB (Factor=0.86) → safe
Snapshot 10: Danger=0.29, Loudness=84.8dB (Factor=0.71) → safe
Snapshot 11: Danger=0.59, Loudness=81.2dB (Factor=0.68) → safe
Snapshot 12: Danger=0.55, Loudness=61.4dB (Factor=0.51) → safe
Snapshot 13: Danger=0.61, Loudness=75.0dB (Factor=0.62) → safe
Snapshot 14: Danger=0.46, Loudness=68.5dB (Factor=0.57) → safe
Snapshot 